## Đọc dataframe và xóa bỏ các trùng lặp


In [121]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
from module.module import chuyen_chieu_ngang_hoac_chieu_dai_sang_m,chuyen_dien_tich_sang_m2,chuyen_hoa,chuyen_hoa_loai_hinh_dat,chuyen_hoa_giay_to_phap_ly,chuyen_hoa_huong_dat,convert_to_trieu,tach_dia_chi,so_hoa_thanh_pho,so_hoa_quan,so_hoa_phuong,apply_so_hoa_phuong


In [122]:
# Đọc file JSON vào DataFrame
df = pd.read_json('../data/data.json')
# Xóa các dòng trùng lặp
df = df.drop_duplicates()

# Đánh số thứ tự lại (reset index)
df = df.reset_index(drop=True)




### Kiểm tra xem có cột nào bị rỗng không

In [123]:
na_full_cols = df.columns[df.isnull().all()]
print("Các cột toàn bộ là NaN:", na_full_cols.tolist())


Các cột toàn bộ là NaN: []


In [124]:
print(df.shape)


(16991, 14)


In [125]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16991 entries, 0 to 16990
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DiaChi           16990 non-null  object 
 1   LoaiHinhDat      16990 non-null  object 
 2   DienTichDat      16990 non-null  object 
 3   Gia/M2           16971 non-null  object 
 4   GiayToPhapLy     16975 non-null  object 
 5   ChieuNgang       14240 non-null  object 
 6   ChieuDai         13152 non-null  object 
 7   DonVi(M2/Hecta)  10117 non-null  object 
 8   HuongDat         6206 non-null   object 
 9   DacDiemNha/Dat   13352 non-null  object 
 10  TenPhanKhu       1297 non-null   object 
 11  MaLo             189 non-null    object 
 12  Gia/Hecta        19 non-null     object 
 13  TenDuong         4 non-null      float64
dtypes: float64(1), object(13)
memory usage: 1.8+ MB
None


Sau khi kiểm tra thì thấy mặc dù chỉ có 16990 dòng mà các cột TenPhanKhu, MaLo, Gia/Hecta, TenDuong gần như 90% đến ngưỡng 100% là giá trị null nên sẽ drop các cột này

In [126]:
df = df.drop(columns=['TenPhanKhu', 'MaLo', 'Gia/Hecta', 'TenDuong','DonVi(M2/Hecta)'])


In [127]:
df.head(5)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",Đất thổ cư,100 m²,"6,50 triệu/m²",Đã có sổ,5 m,20 m,NaN,NaN
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",Đất thổ cư,171 m²,"178,36 triệu/m²",Đã có sổ,9 m,19 m,NaN,NaN
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",Đất thổ cư,90 m²,"21,44 triệu/m²",Đã có sổ,5 m,18 m,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",Đất thổ cư,123 m²,"22,76 triệu/m²",Đã có sổ,4.1 m,NaN,NaN,NaN
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",Đất thổ cư,109 m²,"10,55 triệu/m²",Đã có sổ,5.5 m,20 m,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ


# Chuẩn hóa dữ liệu

## Chuẩn hóa các cột số trước

### Diện Tích Đất

In [128]:
#1 hecta = 10000 mét vuông
# Trích đơn vị bằng cách tách sau dấu cách cuối cùng
df['DonVi_DienTich'] = df['DienTichDat'].astype(str).apply(lambda x: x.split()[-1] if ' ' in x else 'Không rõ')

# Xem tất cả đơn vị xuất hiện
print(df['DonVi_DienTich'].value_counts())
df_khong_ro = df[df['DonVi_DienTich'] == 'Không rõ']
df_hecta = df[df['DonVi_DienTich'] == 'hecta']

print("Các dòng có đơn vị không rõ trong 'DienTichDat':")
print(df_khong_ro[['DienTichDat']])

print("Các dòng có đơn vị không rõ trong 'DienTichDat':")
print(df_hecta[['DienTichDat']])




DonVi_DienTich
m²          16971
hecta          19
Không rõ        1
Name: count, dtype: int64
Các dòng có đơn vị không rõ trong 'DienTichDat':
   DienTichDat
37         NaN
Các dòng có đơn vị không rõ trong 'DienTichDat':
        DienTichDat
331       1.5 hecta
460         4 hecta
1251        9 hecta
2879      9.8 hecta
3180        1 hecta
3807      4.2 hecta
5508   100000 hecta
6162    23000 hecta
6992       60 hecta
7002        5 hecta
7337      2.8 hecta
7773        7 hecta
9570      4.7 hecta
9614        1 hecta
10272     2.4 hecta
10629     6.5 hecta
11003     4.1 hecta
12218     1.4 hecta
15356    2.34 hecta


Có thể thấy là trong cột Diện Tích Đất thì chỉ có 2 dữ liệu là m² và hecta, còn Không rõ là cột bị Nan nên sẽ drop cột đó luôn

In [129]:
df = df.dropna(subset=['DienTichDat'])
print(f"Số dòng còn lại sau khi xoá: {len(df)}")
print(df['DienTichDat'].isnull().sum())  # Kiểm tra còn NaN không


Số dòng còn lại sau khi xoá: 16990
0


In [130]:
df['DienTichDat'] = df['DienTichDat'].apply(chuyen_dien_tich_sang_m2)


### Chiều ngang

In [131]:
print(df['ChieuNgang'].isnull().sum())  # Kiểm tra còn NaN không

2750


In [132]:
df['ChieuNgang_m'] = df['ChieuNgang'].astype(str).apply(lambda x: x.split()[-1] if ' ' in x else 'Không rõ')

# Xem tất cả đơn vị xuất hiện
print(df['ChieuNgang_m'].value_counts())


ChieuNgang_m
m           14240
Không rõ     2750
Name: count, dtype: int64


Cột Chiều Ngang chỉ có 2 trạng thái dữ liệu là m và Null, gán null về không rõ

In [133]:
df["ChieuNgang"]=df["ChieuNgang"].apply(chuyen_chieu_ngang_hoac_chieu_dai_sang_m)

In [134]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat,DonVi_DienTich,ChieuNgang_m
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",Đất thổ cư,100.0,"6,50 triệu/m²",Đã có sổ,5.0,20 m,NaN,NaN,m²,m
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",Đất thổ cư,171.0,"178,36 triệu/m²",Đã có sổ,9.0,19 m,NaN,NaN,m²,m
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",Đất thổ cư,90.0,"21,44 triệu/m²",Đã có sổ,5.0,18 m,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ,m²,m
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",Đất thổ cư,123.0,"22,76 triệu/m²",Đã có sổ,4.1,NaN,NaN,NaN,m²,m
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",Đất thổ cư,109.0,"10,55 triệu/m²",Đã có sổ,5.5,20 m,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ,m²,m
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",Đất nông nghiệp,1500.0,350.000 đ/m²,Đã có sổ,23.0,67 m,Đông Nam,Mặt tiền,m²,m
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",Đất thổ cư,1000.0,"10,50 triệu/m²",Đã có sổ,20.0,50 m,NaN,Hẻm xe hơi,m²,m
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",Đất thổ cư,350.0,"1,14 triệu/m²",Đã có sổ,5.0,70 m,Tây,Mặt tiền,m²,m
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",Đất thổ cư,256.0,"24,22 triệu/m²",Đã có sổ,15.5,15 m,Bắc,Nở hậu\nHẻm xe hơi\nThổ cư toàn bộ,m²,m
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",Đất thổ cư,130.0,20 triệu/m²,Đã có sổ,5.0,27 m,Đông,Mặt tiền\nNở hậu\nThổ cư toàn bộ,m²,m


Chiều dài

In [135]:
print(df['ChieuDai'].isnull().sum())  # Kiểm tra còn NaN không

3838


In [136]:
df['ChieuDai_m'] = df['ChieuDai'].astype(str).apply(lambda x: x.split()[-1] if ' ' in x else 'Không rõ')

# Xem tất cả đơn vị xuất hiện
print(df['ChieuDai_m'].value_counts())

ChieuDai_m
m           13152
Không rõ     3838
Name: count, dtype: int64


Số dòng Nan trùng với số dòng không rõ

In [137]:
df["ChieuDai"]=df["ChieuDai"].apply(chuyen_chieu_ngang_hoac_chieu_dai_sang_m)

In [138]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat,DonVi_DienTich,ChieuNgang_m,ChieuDai_m
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",Đất thổ cư,100.0,"6,50 triệu/m²",Đã có sổ,5.0,20.0,NaN,NaN,m²,m,m
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",Đất thổ cư,171.0,"178,36 triệu/m²",Đã có sổ,9.0,19.0,NaN,NaN,m²,m,m
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",Đất thổ cư,90.0,"21,44 triệu/m²",Đã có sổ,5.0,18.0,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ,m²,m,m
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",Đất thổ cư,123.0,"22,76 triệu/m²",Đã có sổ,4.1,NaN,NaN,NaN,m²,m,Không rõ
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",Đất thổ cư,109.0,"10,55 triệu/m²",Đã có sổ,5.5,20.0,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ,m²,m,m
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",Đất nông nghiệp,1500.0,350.000 đ/m²,Đã có sổ,23.0,67.0,Đông Nam,Mặt tiền,m²,m,m
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",Đất thổ cư,1000.0,"10,50 triệu/m²",Đã có sổ,20.0,50.0,NaN,Hẻm xe hơi,m²,m,m
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",Đất thổ cư,350.0,"1,14 triệu/m²",Đã có sổ,5.0,70.0,Tây,Mặt tiền,m²,m,m
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",Đất thổ cư,256.0,"24,22 triệu/m²",Đã có sổ,15.5,15.0,Bắc,Nở hậu\nHẻm xe hơi\nThổ cư toàn bộ,m²,m,m
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",Đất thổ cư,130.0,20 triệu/m²,Đã có sổ,5.0,27.0,Đông,Mặt tiền\nNở hậu\nThổ cư toàn bộ,m²,m,m


## **Cách xử lý dữ liệu chiều dài và chiều ngang**

### **1. Trường hợp chiều dài và chiều ngang đều thiếu (`NaN`):**
- Nếu cả **chiều dài** và **chiều ngang** đều không có dữ liệu (NaN), thì:
  - **Lấy diện tích đất chia cho 2**.

### **2. Trường hợp có chiều dài nhưng không có chiều ngang:**
- Nếu chỉ có **chiều dài**, thì:
  - **Lấy diện tích đất chia cho chiều dài**.

### **3. Trường hợp có chiều ngang nhưng không có chiều dài:**
- Nếu chỉ có **chiều ngang**, thì:
  - **Lấy diện tích đất chia cho chiều ngang**.


In [139]:
# Áp dụng hàm cho từng dòng trong DataFrame
df[['ChieuDai', 'ChieuNgang']] = df.apply(chuyen_hoa, axis=1)

In [140]:
print("Số cột của chiều dài còn nan:",df['ChieuDai'].isnull().sum())  # Kiểm tra còn NaN không
print("Số cột của chiều ngang còn nan:",df['ChieuNgang'].isnull().sum())  # Kiểm tra còn NaN không

Số cột của chiều dài còn nan: 0
Số cột của chiều ngang còn nan: 0


In [141]:
df = df.drop(columns=['DonVi_DienTich', 'ChieuNgang_m', 'ChieuDai_m'])


In [142]:
df.head()

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",Đất thổ cư,100.0,"6,50 triệu/m²",Đã có sổ,5.0,20.0,NaN,NaN
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",Đất thổ cư,171.0,"178,36 triệu/m²",Đã có sổ,9.0,19.0,NaN,NaN
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",Đất thổ cư,90.0,"21,44 triệu/m²",Đã có sổ,5.0,18.0,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",Đất thổ cư,123.0,"22,76 triệu/m²",Đã có sổ,4.1,30.0,NaN,NaN
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",Đất thổ cư,109.0,"10,55 triệu/m²",Đã có sổ,5.5,20.0,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ


## **Loại hình đất**

In [143]:
print(df['LoaiHinhDat'].value_counts())

LoaiHinhDat
Đất thổ cư         14170
Đất nông nghiệp     1497
Đất nền dự án       1189
Đất công nghiệp      134
Name: count, dtype: int64


In [144]:
print(df['LoaiHinhDat'].isnull().sum())  # Kiểm tra còn NaN không

0


In [145]:
df["LoaiHinhDat"]=df["LoaiHinhDat"].apply(chuyen_hoa_loai_hinh_dat)

In [146]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",0,100.0,"6,50 triệu/m²",Đã có sổ,5.0,20.0,NaN,NaN
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",0,171.0,"178,36 triệu/m²",Đã có sổ,9.0,19.0,NaN,NaN
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",0,90.0,"21,44 triệu/m²",Đã có sổ,5.0,18.0,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",0,123.0,"22,76 triệu/m²",Đã có sổ,4.1,30.0,NaN,NaN
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",0,109.0,"10,55 triệu/m²",Đã có sổ,5.5,20.0,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",1,1500.0,350.000 đ/m²,Đã có sổ,23.0,67.0,Đông Nam,Mặt tiền
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",0,1000.0,"10,50 triệu/m²",Đã có sổ,20.0,50.0,NaN,Hẻm xe hơi
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",0,350.0,"1,14 triệu/m²",Đã có sổ,5.0,70.0,Tây,Mặt tiền
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",0,256.0,"24,22 triệu/m²",Đã có sổ,15.5,15.0,Bắc,Nở hậu\nHẻm xe hơi\nThổ cư toàn bộ
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",0,130.0,20 triệu/m²,Đã có sổ,5.0,27.0,Đông,Mặt tiền\nNở hậu\nThổ cư toàn bộ


### **Đất thổ cư**: 0
### **Đất nông nghiệp**: 1
### **Đất nền dự án**: 2
### **Đất công nghiệp**: 3

## **Giấy tờ pháp lý**

In [147]:
print(df['GiayToPhapLy'].value_counts())

GiayToPhapLy
Đã có sổ                         16544
Đang chờ sổ                        259
Sổ chung / công chứng vi bằng      103
Giấy tờ viết tay                    55
Không có sổ                         11
3                                    3
Name: count, dtype: int64


In [148]:
print(df['LoaiHinhDat'].isnull().sum())  # Kiểm tra còn NaN không

0


In [149]:
df['GiayToPhapLy'] = df['GiayToPhapLy'].apply(chuyen_hoa_giay_to_phap_ly).astype('Int64')


In [150]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",0,100.0,"6,50 triệu/m²",0,5.0,20.0,NaN,NaN
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",0,171.0,"178,36 triệu/m²",0,9.0,19.0,NaN,NaN
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",0,90.0,"21,44 triệu/m²",0,5.0,18.0,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",0,123.0,"22,76 triệu/m²",0,4.1,30.0,NaN,NaN
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",0,109.0,"10,55 triệu/m²",0,5.5,20.0,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",1,1500.0,350.000 đ/m²,0,23.0,67.0,Đông Nam,Mặt tiền
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",0,1000.0,"10,50 triệu/m²",0,20.0,50.0,NaN,Hẻm xe hơi
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",0,350.0,"1,14 triệu/m²",0,5.0,70.0,Tây,Mặt tiền
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",0,256.0,"24,22 triệu/m²",0,15.5,15.0,Bắc,Nở hậu\nHẻm xe hơi\nThổ cư toàn bộ
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",0,130.0,20 triệu/m²,0,5.0,27.0,Đông,Mặt tiền\nNở hậu\nThổ cư toàn bộ


In [151]:
print(df['GiayToPhapLy'].value_counts())

GiayToPhapLy
0    16547
1      259
2      103
3       55
4       11
Name: count, dtype: Int64


#### Thấy có giá trị 3 chỉ có xuất hiện 3 lần nên sẽ chuyển qua đã có sổ luôn

## **Hướng Đất**

In [152]:
print(df['HuongDat'].value_counts())

HuongDat
Đông Nam    1273
Đông Bắc     847
Tây Bắc      818
Tây Nam      762
Đông         725
Nam          633
Tây          600
Bắc          548
Name: count, dtype: int64


In [153]:
print(df['HuongDat'].isnull().sum())  # Kiểm tra còn NaN không

10784


In [154]:
df["HuongDat"]=df["HuongDat"].apply(chuyen_hoa_huong_dat).astype("Int32")

In [155]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",0,100.0,"6,50 triệu/m²",0,5.0,20.0,0,NaN
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",0,171.0,"178,36 triệu/m²",0,9.0,19.0,0,NaN
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",0,90.0,"21,44 triệu/m²",0,5.0,18.0,2,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",0,123.0,"22,76 triệu/m²",0,4.1,30.0,0,NaN
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",0,109.0,"10,55 triệu/m²",0,5.5,20.0,1,Mặt tiền\nNở hậu\nThổ cư toàn bộ
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",1,1500.0,350.000 đ/m²,0,23.0,67.0,1,Mặt tiền
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",0,1000.0,"10,50 triệu/m²",0,20.0,50.0,0,Hẻm xe hơi
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",0,350.0,"1,14 triệu/m²",0,5.0,70.0,7,Mặt tiền
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",0,256.0,"24,22 triệu/m²",0,15.5,15.0,8,Nở hậu\nHẻm xe hơi\nThổ cư toàn bộ
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",0,130.0,20 triệu/m²,0,5.0,27.0,5,Mặt tiền\nNở hậu\nThổ cư toàn bộ


### **Nan hoặc dữ liệu khác**: 0
### **Đông Nam**: 1
### **Đông Bắc**: 2
### **Tây Bắc**: 3
### **Tây Nam**: 4
### **Đông**: 5
### **Nam**: 6
### **Tây**: 7
### **Bắc**: 8

## **Đặc điểm nhà đất**

In [156]:
print(df['DacDiemNha/Dat'].value_counts())

DacDiemNha/Dat
Mặt tiền                                                                        4477
Hẻm xe hơi                                                                      2089
Mặt tiền\nThổ cư toàn bộ                                                        1074
Hẻm xe hơi\nThổ cư toàn bộ                                                       585
Mặt tiền\nThổ cư 1 phần                                                          540
                                                                                ... 
Mặt tiền\nHẻm xe hơi\nChưa có thổ cư\nNở hậu\nThổ cư 1 phần\nKhông có thổ cư       1
Nở hậu\nHẻm xe hơi\nMặt tiền\nThổ cư toàn bộ                                       1
Mặt tiền\nChưa có thổ cư\nThổ cư 1 phần\nNở hậu\nHẻm xe hơi                        1
Mặt tiền\nHẻm xe hơi\nChưa có thổ cư\nThổ cư 1 phần                                1
Không có thổ cư\nChưa có thổ cư                                                    1
Name: count, Length: 190, dtype: int64


In [157]:
print(df['DacDiemNha/Dat'].isnull().sum())  # Kiểm tra còn NaN không

3638


In [158]:
encoder = LabelEncoder()
df['DacDiemNha/Dat'] = encoder.fit_transform(df['DacDiemNha/Dat'])

In [159]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",0,100.0,"6,50 triệu/m²",0,5.0,20.0,0,190
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",0,171.0,"178,36 triệu/m²",0,9.0,19.0,0,190
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",0,90.0,"21,44 triệu/m²",0,5.0,18.0,2,23
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",0,123.0,"22,76 triệu/m²",0,4.1,30.0,0,190
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",0,109.0,"10,55 triệu/m²",0,5.5,20.0,1,109
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",1,1500.0,350.000 đ/m²,0,23.0,67.0,1,54
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",0,1000.0,"10,50 triệu/m²",0,20.0,50.0,0,10
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",0,350.0,"1,14 triệu/m²",0,5.0,70.0,7,54
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",0,256.0,"24,22 triệu/m²",0,15.5,15.0,8,136
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",0,130.0,20 triệu/m²,0,5.0,27.0,5,109


## **Giá/M2**

In [160]:
print(df['Gia/M2'].isnull().sum())  # Kiểm tra còn NaN không


19


In [161]:
df = df.dropna(subset=['Gia/M2'])


In [162]:
df['DonVi_DienTich'] = df['Gia/M2'].str.split('/').str[1]
unique_values = df['DonVi_DienTich'].unique()
print(unique_values)



['m²']


### chỉ có 1 dữ liệu diện tích là m2, không có hecta nên là có thể bỏ được

In [163]:
df['DonVi_GiaCa'] = df['Gia/M2'].str.split('/').str[0].str.split().str[1]

# In các giá trị duy nhất trong cột 'Gia'
unique_values = df['DonVi_GiaCa'].unique()

# In kết quả
print(unique_values)

['triệu' 'đ' 'tỷ']


#### Sẽ dựa giá/m2 là triệu

In [164]:
df['Gia/M2'] = df['Gia/M2'].apply(convert_to_trieu).astype("float")


In [165]:
df=df.drop(columns=["DonVi_DienTich","DonVi_GiaCa"])

In [166]:
df.head(10)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",0,100.0,6.50000,0,5.0,20.0,0,190
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",0,171.0,178.36000,0,9.0,19.0,0,190
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",0,90.0,21.44000,0,5.0,18.0,2,23
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",0,123.0,22.76000,0,4.1,30.0,0,190
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",0,109.0,10.55000,0,5.5,20.0,1,109
5,"153, Tân long, Xã Hòa Thành, Huyện Lai Vung, Đ...",1,1500.0,0.00035,0,23.0,67.0,1,54
6,"Đường Bình Mỹ, Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ...",0,1000.0,10.50000,0,20.0,50.0,0,10
7,"chánh phu hoa 009, Phường Chánh Phú Hòa, Thị x...",0,350.0,1.14000,0,5.0,70.0,7,54
8,"Hợp Nhất, QL6, Xã Đông Phương Yên, Huyện Chươn...",0,256.0,24.22000,0,15.5,15.0,8,136
9,"Đường Số 9, Xã Tân Thông Hội, Huyện Củ Chi, Tp...",0,130.0,20.00000,0,5.0,27.0,5,109


## **Địa chỉ**

In [167]:
df[['Thành Phố / Tỉnh', 'Quận / Huyện', 'Phường / Xã', 'Thông Tin Thêm']] = df['DiaChi'].apply(tach_dia_chi).apply(pd.Series)


In [168]:
df['Mã Tỉnh'] = df.apply(lambda row: so_hoa_thanh_pho(row)['Mã Tỉnh'], axis=1)


In [169]:
df['Mã Quận Huyện'] = df.apply(lambda row: so_hoa_quan(row['Mã Tỉnh'], row['Quận / Huyện']), axis=1)
df['Mã Phường'] = df.apply(lambda row: so_hoa_phuong(row['Mã Tỉnh'], row['Mã Quận Huyện'], row['Phường / Xã']), axis=1)


In [170]:
# Lọc các dòng có "Thừa Thiên Huế" và "Thành phố Huế"
df_filtered = df[(df['Thành Phố / Tỉnh'] == 'Thừa Thiên Huế') & (df['Quận / Huyện'] == 'Thành phố Huế')]

# Áp dụng apply chỉ cho các dòng đã lọc
df_filtered[['Mã Phường', 'Mã Quận Huyện']] = df_filtered.apply(apply_so_hoa_phuong, axis=1)

# Cập nhật lại dữ liệu vào DataFrame ban đầu
df.update(df_filtered)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11364\842856576.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[['Mã Phường', 'Mã Quận Huyện']] = df_filtered.apply(apply_so_hoa_phuong, axis=1)


In [171]:
df = df.drop(columns=["DiaChi","Thành Phố / Tỉnh", "Quận / Huyện", "Phường / Xã", "Thông Tin Thêm"])


In [172]:
df.to_json("../data/real-estate.json")
